# Fine-tune LLM with PyTorch FSDP and QLora on Amazon SageMaker AI using ModelTrainer

In this notebook, we fine-tune LLM on Amazon SageMaker AI, using Python scripts and SageMaker ModelTrainer for executing a training job.

## Prerequisites

In [ ]:
%pip install -r ./scripts/requirements.txt --upgrade

***

## Setup Configuration file path

If you have created a Managed MLflow server, copy the `ARN` code here and assign a name to the experiment

In [ ]:
import os

model_id = "meta-llama/Llama-3.1-8B-Instruct"

os.environ["HF_TOKEN"] = "<HF_TOKEN>"
os.environ["model_id"] = model_id
os.environ["mlflow_uri"] = ""
os.environ["mlflow_experiment_name"] = "llama-31-8b-sft"

***

## Visualize and upload the dataset

We are going to load [UCSC-VLAA/MedReason](https://huggingface.co/datasets/UCSC-VLAA/MedReason) dataset

In [ ]:
from sagemaker.core.helper.session_helper import Session

In [ ]:
sagemaker_session = Session()
bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix

In [ ]:
from datasets import load_dataset

dataset = load_dataset("UCSC-VLAA/MedReason", split="train[:10000]")

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)

df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size=0.1, random_state=42)
train, test = train_test_split(train, test_size=10, random_state=42)

print("Number of train elements: ", len(train))
print("Number of test elements: ", len(val))
print("Number of test elements: ", len(test))

Create a prompt template and load the dataset with a random sample to try summarization.

In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_id)
# Define EOS token
tokenizer.eos_token = "<|eot_id|>"
tokenizer.eos_token_id = 128009

# Define PAD token
tokenizer.pad_token = tokenizer.eos_token


def prepare_dataset(sample):

    system_text = (
        "You are a deep-thinking AI assistant.\n\n"
        "For every user question, first write your thoughts and reasoning inside <think>...</think> tags, then provide your answer."
    )

    messages = []

    messages.append({"role": "system", "content": system_text})
    messages.append({"role": "user", "content": sample["question"]})
    messages.append(
        {
            "role": "assistant",
            "content": f"<think>\n{sample['reasoning']}\n</think>\n{sample['answer']}",
        }
    )

    # Apply chat template
    sample["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return sample

In [ ]:
from datasets import Dataset, DatasetDict
from random import randint

train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)
test_dataset = Dataset.from_pandas(test)

dataset = DatasetDict({"train": train_dataset, "val": val_dataset})

train_dataset = dataset["train"].map(
    prepare_dataset, remove_columns=list(train_dataset.features)
)

print(train_dataset[randint(0, len(dataset))]["text"])

val_dataset = dataset["val"].map(
    prepare_dataset, remove_columns=list(val_dataset.features)
)

### Upload to Amazon S3

In [ ]:
import boto3
import shutil
from sagemaker.core.helper.session_helper import Session

In [ ]:
sagemaker_session = Session()
s3_client = boto3.client('s3')

bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix

In [ ]:
# save train_dataset to s3 using our SageMaker session
if default_prefix:
    input_path = f"{default_prefix}/datasets/llm-fine-tuning-modeltrainer-sft"
else:
    input_path = f"datasets/llm-fine-tuning-modeltrainer-sft"

train_dataset_s3_path = f"s3://{bucket_name}/{input_path}/train/dataset.json"
val_dataset_s3_path = f"s3://{bucket_name}/{input_path}/val/dataset.json"

In [ ]:
# Save datasets to s3
# We will fine tune only with 20 records due to limited compute resource for the workshop
train_dataset.to_json("./data/train/dataset.json", orient="records")
val_dataset.to_json("./data/val/dataset.json", orient="records")

s3_client.upload_file("./data/train/dataset.json", bucket_name, f"{input_path}/train/dataset.json")
s3_client.upload_file("./data/val/dataset.json", bucket_name, f"{input_path}/val/dataset.json")

shutil.rmtree("./data")

print(f"Training data uploaded to:")
print(train_dataset_s3_path)
print(val_dataset_s3_path)

***

## Model fine-tuning

We are now ready to fine-tune our model. We will use the [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) from transfomers to fine-tune our model. We prepared a script [train.py](./scripts/train.py) which will loads the dataset from disk, prepare the model, tokenizer and start the training.

For configuration we use `TrlParser`, that allows us to provide hyperparameters in a `yaml` file. This yaml will be uploaded and provided to Amazon SageMaker similar to our datasets. We are saving the config file as `args.yaml` and upload it to S3.

In [ ]:
%%bash

cat > ./args.yaml <<EOF
model_id: "${model_id}"                           # Hugging Face model id
mlflow_uri: "${mlflow_uri}"                       # MLflow tracking server URI
mlflow_experiment_name: "${mlflow_experiment_name}" # MLflow experiment name
# sagemaker specific parameters
output_dir: "/opt/ml/model"                       # path to where SageMaker will upload the model 
checkpoint_dir: "/opt/ml/checkpoints/"            # directory for saving training checkpoints
train_dataset_path: "/opt/ml/input/data/train/"   # path to where S3 saves train dataset
val_dataset_path: "/opt/ml/input/data/val/"       # path to where S3 saves test dataset
token: "${HF_TOKEN}"                              # Hugging Face API token
merge_weights: true                               # merge weights in the base model
# training parameters
apply_truncation: true                           # apply truncation to datasets
attn_implementation: "flash_attention_2"         # attention implementation type
learning_rate: 1e-4                              # learning rate scheduler
num_train_epochs: 10                             # number of training epochs
per_device_train_batch_size: 4                   # batch size per device during training
per_device_eval_batch_size: 2                    # batch size for evaluation
gradient_accumulation_steps: 4                   # number of steps before performing a backward/update pass
gradient_checkpointing: true                     # use gradient checkpointing
torch_dtype: "bfloat16"                          # float precision type
bf16: true                                       # use bfloat16 precision
tf32: true                                       # use tf32 precision
ignore_data_skip: true                           # skip data loading errors
logging_strategy: "steps"                        # logging strategy
logging_steps: 1                                 # log every N steps
log_on_each_node: false                          # disable logging on each node
ddp_find_unused_parameters: false                # DDP unused parameter detection
save_total_limit: 1                              # maximum number of checkpoints to keep
save_steps: 100                                  # Save checkpoint every this many steps
warmup_steps: 100                                # number of warmup steps
weight_decay: 0.01                               # weight decay coefficient
fsdp: "full_shard auto_wrap offload"             # FSDP sharding strategy
fsdp_config:                                     # FSDP configuration options
    backward_prefetch: "backward_pre"            # prefetch parameters during backward pass
    cpu_ram_efficient_loading: true              # enable CPU RAM efficient model loading
    offload_params: true                         # offload parameters to CPU
    forward_prefetch: false                      # disable forward prefetch
    use_orig_params: false                       # use original parameter names
# LoRA parameters
load_in_4bit: true                               # enable 4-bit quantization
lora_r: 32                                       # LoRA rank
lora_alpha: 64                                   # LoRA alpha parameter
lora_dropout: 0.05                               # LoRA dropout rate
EOF

Lets upload the config file to S3.

In [ ]:
import os

if default_prefix:
    input_path = f"{default_prefix}/datasets/llm-fine-tuning-modeltrainer-sft"
else:
    input_path = f"datasets/llm-fine-tuning-modeltrainer-sft"

train_config_s3_path = f"s3://{bucket_name}/{input_path}/config/args.yaml"

# upload the model yaml file to s3
model_yaml = "args.yaml"
s3_client.upload_file(model_yaml, bucket_name, f"{input_path}/config/args.yaml")
os.remove("./args.yaml")

print(f"Training config uploaded to:")
print(train_config_s3_path)

## Fine-tune model

Below estimtor will train the model with QLoRA, merge the adapter in the base model and save in S3

#### Get PyTorch image_uri

We are going to use the native PyTorch container image, pre-built for Amazon SageMaker

In [ ]:
from sagemaker.core import image_uris
from sagemaker.core.helper.session_helper import Session

In [ ]:
sagemaker_session = Session()

bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix

In [ ]:
instance_type = "ml.p4d.24xlarge"
instance_count = 1

instance_type

In [ ]:
image_uri = image_uris.retrieve(
    framework="huggingface",
    region=sagemaker_session.boto_session.region_name,
    version="4.56.2",
    base_framework_version="pytorch2.8.0",
    instance_type=instance_type,
    image_scope="training",
)

image_uri

In [ ]:
from sagemaker.train.configs import (
    CheckpointConfig,
    Compute,
    OutputDataConfig,
    SourceCode,
    StoppingCondition,
)
from sagemaker.train.distributed import Torchrun
from sagemaker.train.model_trainer import ModelTrainer

# Define the script to be run
source_code = SourceCode(
    source_dir="./scripts",
    requirements="requirements.txt",
    entry_script="train.py",
)

# Define the compute
compute_configs = Compute(
    instance_type=instance_type,
    instance_count=instance_count,
    keep_alive_period_in_seconds=0,
)

# define Training Job Name
job_name = f"train-{model_id.split('/')[-1].replace('.', '-')}-sft"

# define OutputDataConfig path
if default_prefix:
    output_path = f"s3://{bucket_name}/{default_prefix}/{job_name}"
else:
    output_path = f"s3://{bucket_name}/{job_name}"

# Define the ModelTrainer
model_trainer = ModelTrainer(
    training_image=image_uri,
    source_code=source_code,
    base_job_name=job_name,
    compute=compute_configs,
    distributed=Torchrun(),
    stopping_condition=StoppingCondition(max_runtime_in_seconds=18000),
    hyperparameters={
        "config": "/opt/ml/input/data/config/args.yaml"  # path to TRL config which was uploaded to s3
    },
    output_data_config=OutputDataConfig(s3_output_path=output_path),
    checkpoint_config=CheckpointConfig(
        s3_uri=output_path + "/checkpoint", local_path="/opt/ml/checkpoints"
    ),
)

In [ ]:
from sagemaker.train.configs import InputData

# Pass the input data
train_input = InputData(
    channel_name="train",
    data_source=train_dataset_s3_path, # S3 path where training data is stored
)

val_input = InputData(
    channel_name="val",
    data_source=val_dataset_s3_path, # S3 path where training data is stored
)

config_input = InputData(
    channel_name="config",
    data_source=train_config_s3_path, # S3 path where training data is stored
)

# Check input channels configured
data = [train_input, val_input, config_input]
data

In [ ]:
# starting the train job with our uploaded datasets as input
model_trainer.train(input_data_config=data, wait=False)

***

# Model Deployment

In the following sections, we are going to deploy the fine-tuned model on an Amazon SageMaker Real-time endpoint.

## Load Fine-Tuned model

In [ ]:
import boto3
from sagemaker.core.helper.session_helper import Session

In [ ]:
sagemaker_session = Session()
sm_client = boto3.client("sagemaker")

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"

bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix
job_prefix = f"train-{model_id.split('/')[-1].replace('.', '-')}-sft"

In [ ]:
def get_last_job_name(job_name_prefix):
    sagemaker_client = boto3.client('sagemaker')

    matching_jobs = []
    next_token = None

    while True:
        # Prepare the search parameters
        search_params = {
            'Resource': 'TrainingJob',
            'SearchExpression': {
                'Filters': [
                    {
                        'Name': 'TrainingJobName',
                        'Operator': 'Contains',
                        'Value': job_name_prefix
                    },
                    {
                        'Name': 'TrainingJobStatus',
                        'Operator': 'Equals',
                        'Value': "Completed"
                    }
                ]
            },
            'SortBy': 'CreationTime',
            'SortOrder': 'Descending',
            'MaxResults': 100
        }

        # Add NextToken if we have one
        if next_token:
            search_params['NextToken'] = next_token

        # Make the search request
        search_response = sagemaker_client.search(**search_params)

        # Filter and add matching jobs
        matching_jobs.extend([
            job['TrainingJob']['TrainingJobName'] 
            for job in search_response['Results']
            if job['TrainingJob']['TrainingJobName'].startswith(job_name_prefix)
        ])

        # Check if we have more results to fetch
        next_token = search_response.get('NextToken')
        if not next_token or matching_jobs:  # Stop if we found at least one match or no more results
            break

    if not matching_jobs:
        raise ValueError(f"No completed training jobs found starting with prefix '{job_name_prefix}'")

    return matching_jobs[0]

In [ ]:
job_name = get_last_job_name(job_prefix)

job_name

#### Inference configurations

In [ ]:
from sagemaker.core.helper.session_helper import get_execution_role

In [ ]:
num_accelerators = 4

In [ ]:
region = sagemaker_session.boto_session.region_name
CONTAINER_VERSION = "vllm:0.14.0-gpu-py312-cu129-ubuntu22.04-sagemaker"

image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/{CONTAINER_VERSION}"

image_uri

##### Create the Model

In [ ]:
import json

env = {
    "SM_VLLM_MODEL": "/opt/ml/model",
    "HF_TOKEN": os.environ.get("HF_TOKEN", ""),
    "SM_VLLM_DTYPE": "bfloat16",
    "SM_VLLM_GPU_MEMORY_UTILIZATION": "0.8",
    "SM_VLLM_MAX_MODEL_LEN": json.dumps(1024 * 16),
    "SM_VLLM_MAX_NUM_SEQS": "16",
    "SM_VLLM_ENABLE_CHUNKED_PREFILL": "true",
    "SM_VLLM_KV_CACHE_DTYPE": "auto",
    "SM_VLLM_TENSOR_PARALLEL_SIZE": str(num_accelerators),
}

In [ ]:
model_response = sm_client.create_model(
    ModelName=f"{model_id.split('/')[-1].replace('.', '-')}-model",
    ExecutionRoleArn=get_execution_role(),
    PrimaryContainer={
        "Image": image_uri,
        "Environment": env,
        "ModelDataSource": {
            "S3DataSource": {
                "S3Uri": f"s3://{bucket_name}/{job_prefix}/{job_name}/output/model.tar.gz",
                "S3DataType": "S3Prefix",
                "CompressionType": "Gzip",
            }
        },
    },
)

model_response

##### Create the endpoint config

In [ ]:
instance_count = 1
instance_type = "ml.g5.12xlarge"
health_check_timeout = 700

In [ ]:
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=f"{model_id.split('/')[-1].replace('.', '-')}-config",
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": f"{model_id.split('/')[-1].replace('.', '-')}-model",
            "InstanceType": instance_type,
            "InitialInstanceCount": instance_count,
            "ModelDataDownloadTimeoutInSeconds": health_check_timeout,
            "ContainerStartupHealthCheckTimeoutInSeconds": health_check_timeout,
            "InferenceAmiVersion": "al2-ami-sagemaker-inference-gpu-3-1",
        }
    ],
)

endpoint_config_response

##### Create endpoint

In [ ]:
endpoint_name = f"{model_id.split('/')[-1].replace('.', '-')}-vllm"

In [ ]:
endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=f"{model_id.split('/')[-1].replace('.', '-')}-config",
)

endpoint_response

#### Predict

In [ ]:
import boto3
from sagemaker.core.helper.session_helper import Session

In [ ]:
sagemaker_session = Session()

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"

endpoint_name = f"{model_id.split('/')[-1].replace('.', '-')}-vllm"

In [ ]:
predictor = boto3.client("sagemaker-runtime")

In [ ]:
import json
import pandas as pd

eval_dataset = []

for index, el in enumerate(test_dataset, 1):
    print("Processing item ", index)

    payload = {
        "messages": [
            {
                "role": "system",
                "content": "You are a deep-thinking AI assistant.\n\nFor every user question, first write your thoughts and reasoning inside <think>...</think> tags, then provide your answer.",
            },
            {"role": "user", "content": el["question"]},
        ],
        "max_tokens": 4096,
        "stop": ["<|eot_id|>", "<|end_of_text|>"],
        "temperature": 0.4,
        "top_p": 0.9,
        "repetition_penalty": 1.15,
        "no_repeat_ngram_size": 3,
        "do_sample": True,
    }

    response = predictor.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
    )

    result = json.loads(response["Body"].read().decode())
    eval_dataset.append([el["question"], result["choices"][0]["message"]["content"]])

    print("**********************************************")

eval_dataset_df = pd.DataFrame(
    eval_dataset, columns=["question", "answer"]
)

eval_dataset_df.to_json(
    "./eval_dataset_results.jsonl", orient="records", lines=True
)

#### Delete Endpoint

In [ ]:
import boto3

In [ ]:
sm_client = boto3.client("sagemaker")

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"

endpoint_name = f"{model_id.split('/')[-1].replace('.', '-')}-vllm"

In [ ]:
sm_client.delete_model(ModelName=f"{model_id.split('/')[-1].replace('.', '-')}-model")
sm_client.delete_endpoint_config(
    EndpointConfigName=f"{model_id.split('/')[-1].replace('.', '-')}-config"
)
sm_client.delete_endpoint(EndpointName=endpoint_name)